# EO4SD SHORELINE CHANGE MAPPING AND FORECASTING

This code has been modifed by Carpenter (2020) for the project Earth Observation for Sustainable Development. Below demonstrates an example processing workflow for Benin and Togo's Coastline between 2000-2020.

This software is based on scripts and code developed by:
* Vos K., Splinter K.D., Harley M.D., Simmons J.A., Turner I.L. (2019). CoastSat: a Google Earth Engine-enabled Python toolkit to extract shorelines from publicly available satellite imagery. Environmental Modelling and Software. 122, 104528. https://doi.org/10.1016/j.envsoft.2019.104528

It enables the users to extract time-series of shoreline change over the last 20+ years at their site of interest.
There are three main steps:
1. Retrieval of median composite satellite images of the region of interest from Google Earth Engine
2. Shoreline extraction at sub-pixel resolution

## Initial settings

Refer to the Set-up and Installation section of the User Handbook for instructions on how to install the Python packages necessary to run the software, including Google Earth Engine Python API. See original methodology via https://github.com/kvos/CoastSat

In [3]:
import os
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from utils.print_utils import printProgress, printWarning
from coastsat import NOC_tools, NOC_shoreline, NOC_download, NOC_preprocess, \
    SDS_tools, SDS_shoreline, SDS_download, SDS_preprocess

settings ={'output_epsg': 32620,
           
           'cloud_thresh':0.9,
           
            # [ONLY FOR ADVANCED USERS] shoreline detection parameters:
            'min_beach_area': 4500,     # minimum area (in metres^2) for an object to be labelled as a beach
            'buffer_size': 150,         # radius (in metres) for buffer around sandy pixels considered in the shoreline detection
            'min_length_sl': 200,       # minimum length (in metres) of shoreline perimeter to be valid
            'cloud_mask_issue': False,  # switch this parameter to True if sand pixels are masked (in black) on many images  
 
            ## Image download Parameters\n",
            # Landsat\n",
            'LCloudScore': 15,         # Mean cloud score threshold (include images with less then threshold)\n",
            'add_L7_to_L5': True,      # Add Landsat 7 to Landsat 5 median composite if they are in same time period\n",
            'add_L5_to_L7': True,      # Add Landsat 5 to Landsat 7 median composite if they are in same time period\n",
            'add_L7_to_L8': False,      # Add Landsat 7 to Landsat 8 median composite if they are in same time period\n",
            'LCloudThreshold': 35,     # Pixels from a single image in a collection larger than this cloud score threshold\n",
                                       # will be masked.\n",
            'sand_color': '',
           
           'coregistration': False,
           
            # Sentinel
            'CLOUD_FILTER': 50,         # [Integer] Maximum image cloud cover percent allowed in image collection'
            'CLD_PRB_THRESH': 25,       # {Integer] Cloud probability (%); values greater than are considered cloud
            'NIR_DRK_THRESH': 0.08,     # [Float] Near-infrared reflectance; values less than are considered potential cloud shadow
            'CLD_PRJ_DIST': 2,          # [Float] Maximum distance (km) to search for cloud shadows from cloud edges |
            'BUFFER': 50,               # [Integer] Distance (m) to dilate the edge of cloud-identified objects |

            'reference_shoreline': True,
            'max_dist_ref': 100,        # max distance (in meters) allowed from the reference shoreline
   
           # labels for classes for the classifier and colours for display
           'classes':{'hard': [1, [1, 0, 0]], 'nature': [2, [0, 1, 0]],
                      'urban': [3, [1, 1, 0]], 'white-water': [4, [1, 0, 1]],
                      'water': [5, [0, 0.4, 1]], 'sand': [6, [1, 0.6, 0]]},
           
            'bands': {'L8': {'pan': [['B8'],15],
                             'ms':  [['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'BQA'], 30]},
                      'S2': {'10m': [['B2', 'B3', 'B4', 'B8'], 10],
                             '20m': [['B5', 'B6', 'B7', 'B8A', 'B11', 'B12'], 20],
                             '60m': [['B12'], 60]}, }
          }
# directory where the data will be accessed and stored
data_partition = 'c:\\data\\coastsat'
country = 'UK'
base_dir_path = os.path.join(data_partition, country)  
sites_dir_path = os.path.join(base_dir_path, 'sites')
sites = os.listdir(sites_dir_path)

dates = [['2019-01-01', '2019-06-30'],
         ['2019-07-01', '2019-12-31'],
         ['2020-01-01', '2020-06-30'],
         ['2020-07-01', '2020-12-31']]

inputs = {'sat_name': 'S2'}

## download optical data from GEE

In [ ]:
for site in sites:
    for date_pair in dates:

        kml_filepath = os.path.join(sites_dir_path, site)
        kml_polygon = NOC_tools.polygon_from_kml(kml_filepath)
        roi_polygon = SDS_tools.smallest_rectangle(kml_polygon)

        site_name = site[:site.find('.')]
        median_dir_path = os.path.join(base_dir_path, site_name, 'median')
        inputs['site_name'] = site_name
        inputs['dates'] = date_pair 
        inputs['median_dir_path'] = median_dir_path
        settings['inputs'] = inputs
        
        printProgress(f'processing {site_name}: {date_pair}')
        
        NOC_download.retrieve_median_optical(settings)
    
metadata = NOC_download.save_metadata(settings)

## find reference threshold

In [6]:
inputs['pansharpen'] = False
inputs['reference_threshold'] = 0
inputs['dates'] = ['2019-01-01', '2020-12-31']
    
for site in sites:
    
    site_name = site[:site.find('.')]
    median_dir_path = os.path.join(base_dir_path, site_name, 'median')
    inputs['site_name'] = site_name
    inputs['median_dir_path'] = median_dir_path
    settings['inputs'] = inputs
    
    printProgress(f'processing {site_name}')
    
    %matplotlib qt
    NOC_shoreline.find_reference_threshold(settings)

progress  >>>  processing Weymouth
progress  >>>  file_names loaded
progress  >>>  displaying reference histogram
@@@ MNDWI min: -0.8505577802711499
@@@ MNDWI max: 0.6108129772317995
progress  >>>  shoreline extracted


## extract shorelines

In [8]:
inputs = {'sat_name': sat_name}

inputs['create_reference_shoreline'] = True 
inputs['pansharpen'] = True
inputs['reference_threshold'] = -0.25
        
for site in sites:
    for date_pair in dates:
 
        site_name = site[:site.find('.')]
        median_dir_path = os.path.join(base_dir_path, site_name, 'median')
        inputs['site_name'] = site_name
        inputs['dates'] = date_pair 
        inputs['median_dir_path'] = median_dir_path
        settings['inputs'] = inputs

        printProgress(f'processing {site_name}: {date_pair}')
        
        metadata = NOC_download.load_metadata(settings)
        print(metadata)
        
        %matplotlib qt
        output = NOC_shoreline.extract_shoreline_optical(metadata, settings)

progress  >>>  processing Weymouth: ['2019-01-01', '2019-06-30']
progress  >>>  metadata loaded
{'file_names': ['Weymouth_median_S2019-01-01_E2019-06-30_10m.txt', 'Weymouth_median_S2019-01-01_E2019-06-30_20m.txt', 'Weymouth_median_S2019-01-01_E2019-06-30_60m.txt'], 'epsg': 4326, 'date_start': '2019-01-01', 'date_end': '2019-06-30', 'number_images': 29}


KeyError: 'S2'

In [ ]:
gdf = NOC_tools.output_to_gdf(output, 'lines')

satname = output['satname'][0]
file_string = f'{sitename}_shorelines_{satname}.geojson'

gdf.crs = {'init':'epsg:'+str(settings['output_epsg'])} # set layer projection
# save GEOJSON layer to file
gdf.to_file(os.path.join(inputs['filepath'], inputs['sitename'],
                         file_string),
                         driver='GeoJSON', encoding='utf-8')
print('finished ...')

In [ ]:
fig = plt.figure(figsize=[15,8])
plt.axis('equal')
plt.xlabel('Eastings')
plt.ylabel('Northings')
plt.grid(linestyle=':', color='0.5')
for i in range(len(output['shorelines'])):
    sl = output['shorelines'][i]
    date = output['dates'][i]
    plt.plot(sl[:,0], sl[:,1], '.', label=date.strftime('%d-%m-%Y'))
plt.legend();